In [1]:
from dataset import PolarDecDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch
from models.wrappers.mamba_32bits import MambaPolarDecoder

/home/anjal/Desktop/Code/mamba-seq-pred/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
N = 32
CONFIG_NO = 2

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Dataset

In [4]:
dataset = PolarDecDataset(snr_db=10, num_samples=1000000)
test_set = PolarDecDataset(snr_db=10, num_samples=1000000)


In [5]:
train_dataloader = DataLoader(dataset, batch_size = 32)
test_dataloader = DataLoader(test_set, batch_size = 32)

## Model

In [6]:
model = MambaPolarDecoder(
    d_model=64,
    num_layer_encoder=1,
    num_layers_bimamba_block=32,
    seq_len=N,
    d_state=32,
    d_conv=4,
    expand=2
).to(device)
model

MambaPolarDecoder(
  (discrete_embedding): Embedding(2, 64)
  (linear_embedding1): Linear(in_features=1, out_features=64, bias=True)
  (linear_embedding2): Linear(in_features=1, out_features=64, bias=True)
  (linear_input_layer): Linear(in_features=192, out_features=64, bias=True)
  (encoder_layers): ModuleList(
    (0): BiMambaEncoder(
      (layers): ModuleList(
        (0-31): 32 x BiMambaBlock(
          (pre_ln_f): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (mamba_f): Mamba(
            (in_proj): Linear(in_features=64, out_features=256, bias=False)
            (conv1d): Conv1d(128, 128, kernel_size=(4,), stride=(1,), padding=(3,), groups=128)
            (act): SiLU()
            (x_proj): Linear(in_features=128, out_features=68, bias=False)
            (dt_proj): Linear(in_features=4, out_features=128, bias=True)
            (out_proj): Linear(in_features=128, out_features=64, bias=False)
          )
          (post_ln_f): LayerNorm((64,), eps=1e-05, elementw

## Minor modification to the Loss Function: Calculates loss only at non frozen positions

In [7]:
def calculate_loss(frozen_bit_prior, target_vector, predicted_vector,  reliable_only=False):
    """
    frozen_bit_prior: tensor of shape (seq_len,) with 1 for frozen, 0 for message bits
    target_vector: tensor of shape (seq_len,)
    predicted_vector: tensor of shape (seq_len,)
    loss_fn: PyTorch loss function
    """

    if reliable_only: 
     mask = (frozen_bit_prior != 1) 
     target_vector = target_vector[mask]
     predicted_vector = predicted_vector[mask]

    #print("target vector:" ,target_vector[:32], "\n")
    #print("pred vector:" ,predicted_vector[:32])

   # print(f"Length of reliable bits: {len(reliable_target)}")
    loss_fn = torch.nn.BCEWithLogitsLoss()

    return loss_fn(predicted_vector, target_vector)

## Sanity Check

In [8]:
llr, frozen_tensor, snr_tensor, target_tensor= next(iter(train_dataloader))
ip1 = llr.float().to(device)
ip2 = frozen_tensor.int().to(device)
ip3 = snr_tensor.float().to(device)

predicted = model(ip1, ip2, ip3) #works

loss = calculate_loss(ip2, target_tensor.to(device), predicted.to(device)) #works

#print(f"Channel Observation Vector: {channel_tensor}\n\n")


#print(f"Channel Observation Vector: {ip1.shape}\nFrozen Tensor: {ip2.shape}\n")
#print(f"Predicted Channel Input Vector(logits): {predicted.shape}\n\n")

#print(f"Predicted (sigmoid): {torch.sigmoid(predicted)}\n\n")
pred = (torch.sigmoid(predicted) > 0.5).long()[0]

print(f"Predicted bits:{''.join(map(str, pred.cpu().tolist()))}\n")
print(f"Actual bits: {''.join(str(int(i)) for i in target_tensor[0])}\n")
print(f"Loss: {loss}")

Predicted bits:00110100100001000000000000000000

Actual bits: 00011100100000000000000000000000

Loss: 0.44349950551986694


In [9]:

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                        factor=0.5, patience=3)


In [10]:
def train_one_epoch(epoch_index):

    running_loss = 0
    last_loss = 0


    for i, data in enumerate(train_dataloader):
        channel_tensor, frozen_tensor, snr_tensor, target_tensor = data
        ip1 = channel_tensor.float().to(device)
        ip2 = frozen_tensor.int().to(device)
        ip3 = snr_tensor.float().to(device)
        op = target_tensor.to(device)
        optimizer.zero_grad()
        outputs = model(ip1,ip2 ,ip3 ).to(device)

    #    B, L, C = outputs.shape
    #    output_logits = outputs.view(B*L, C).to(device)
    #    target_flattened = shifted.view(B*L).to(device).long()


    #    loss = loss_fn(output_logits, target_flattened)
        
        loss = calculate_loss(ip2, op, outputs)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item()
        if i%1000 == 999:
            last_loss = running_loss/1000
            print('  batch {} loss: {}\n'.format(i + 1, last_loss))
          #  print(f"Predictions look currently like: {outputs[:32]}\n\n")
            running_loss = 0.
    return last_loss


In [11]:
def train( epochs=50):

    best_vloss = 1_000_000.

    for epoch in range(epochs):
     print('EPOCH {}:'.format(epoch + 1))
 
   
     model.train(True)
     avg_loss = train_one_epoch(epoch)


     running_vloss = 0.0
    
     model.eval()

   
     with torch.no_grad():
        for i, vdata in enumerate(test_dataloader):
            vchannel_tensor, vfrozen_tensor, vsnr_tensor, vtarget_tensor = vdata
            voutputs = model(vchannel_tensor.float().to(device), vfrozen_tensor.int().to(device), vsnr_tensor.float().to(device))
          #  B, L, C = voutputs.shape
          #  vloss = loss_fn(voutputs.view(B*L, C).to(device), vlabels.view(B*L).to(device))
            
            vloss = calculate_loss(vfrozen_tensor.to(device), vtarget_tensor.to(device), voutputs.to(device))
            running_vloss += vloss

     avg_vloss = running_vloss / (i + 1)
     print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    
     scheduler.step(avg_vloss)

    
     if avg_vloss < best_vloss:
      
    
      best_vloss = avg_vloss
      model_path = f'./checkpoints/config_{CONFIG_NO}/model_epoch_{epoch}.pt'
      torch.save({
         "comments": "Set the loss function to evaluate loss for both frozen and non frozen bits. set expand=3",
    'model_config': {
        "d_model": model.d_model,
        "num_layer_encoder": model.num_layer_encoder,
        "num_layers_bimamba_block": model.num_layers_bimamba_block,
        "seq_len": model.seq_len,
        "d_state": model.d_state,
        "d_conv": model.d_conv,
        "expand": model.expand,
    },
    'epoch': epoch + 1,
    'train_loss': avg_loss,
    'val_loss': avg_vloss,
    'state_dict': model.state_dict()
}, model_path)


     
    print("Training completed. Model available to use")





In [12]:
train(epochs=30)

EPOCH 1:
  batch 1000 loss: 0.21707879555225373

  batch 2000 loss: 0.1917518671452999

  batch 3000 loss: 0.18240208588540555

  batch 4000 loss: 0.17807611653208733

  batch 5000 loss: 0.17228572613745927

  batch 6000 loss: 0.17099464400857686

  batch 7000 loss: 0.16847753942012786

  batch 8000 loss: 0.16549422273784875

  batch 9000 loss: 0.16474470458179713

  batch 10000 loss: 0.16465505230426788

  batch 11000 loss: 0.1625800811946392

  batch 12000 loss: 0.1610225873440504

  batch 13000 loss: 0.16026050624996424

  batch 14000 loss: 0.15957979682087897

  batch 15000 loss: 0.15892285595089198

  batch 16000 loss: 0.15895515418797732

  batch 17000 loss: 0.15947539995610713

  batch 18000 loss: 0.15894125133752823

  batch 19000 loss: 0.1582379503250122

  batch 20000 loss: 0.15855647423118352

  batch 21000 loss: 0.15941906003654002

  batch 22000 loss: 0.1581791153177619

  batch 23000 loss: 0.1583668457120657

  batch 24000 loss: 0.1573456007912755

  batch 25000 loss: 0.1

KeyboardInterrupt: 